In [130]:
%load_ext autoreload
%autoreload 2  
%reload_ext autoreload
import sys
sys.path.append('..')
import pickle
from pathlib import Path
from rich import print
from rich.console import Console
import pdb
import json
import random
import copy
console = Console()

# Import your modules
from dialop.evaluate import CheckpointManager
from dialop.responses_class import ParallelConversations, ResponseManager, Conversation
from dialop.envs.planning import PlanningEnv
from skills import Agent_tell

print("Imports successful!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Imports successful!

In [43]:

import gspread
from google.oauth2.service_account import Credentials

SERVICE_ACCOUNT_FILE = '/Users/georgiazhou/research_machine/dialop/dialop-8759580d9f40.json'
SCOPE = ["https://spreadsheets.google.com/feeds", 
         "https://www.googleapis.com/auth/drive"]
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPE)
client = gspread.authorize(credentials)
spreadsheet = client.open('Dialop-GPT4-Planning')
worksheet = spreadsheet.get_worksheet(8)

In [98]:
# Initialize checkpoint manager
checkpoint_mgr = CheckpointManager()
print(checkpoint_mgr.checkpoint_dir)

# Load your saved checkpoint
try:
   
    state = checkpoint_mgr.load_full_state('agent_ready_t2')
    
    print("Successfully loaded checkpoint!")
    
    # Access key components
    players = state['players']
    extracted_features = state.get('extracted_features', [])
    t = state.get('t', 0)
    env = state.get('env', None)
    if not env:
        print("No environment found in checkpoint.")
    
    print(f"\nTurn: {t}")
    print(f"Features: {extracted_features}")
    print(f"Players: {list(players.keys())}")
    
except Exception as e:
    print(f"Error loading checkpoint: {str(e)}")

Loaded .api_key


/Users/georgiazhou/research_machine/dialop/dialop/checkpoints/debug_states

Successfully loaded checkpoint!

Turn: 2

Features: ['', '{AtLeastOneEventType: landmark, rating: good, has_parking: true}']

Players: ['agent', 'user']

In [99]:
# Access player objects
agent = players['agent']
user = players['user']


# Check agent's chat history
print("=== Agent Info ===")

chat_history = agent.prompt
print(f"Agent role: {agent.role}")
#print(f"Agent chat history: {chat_history}")

print("\n=== User Info ===")
print(f"User role: {user.role}")
if hasattr(user, 'user_prompt_obss'):
    print(f"User observations: {user.user_prompt_obss}")
    #print(user.prompt)
    


=== Agent Info ===

Agent role: agent

=== User Info ===

User role: user

User observations: 
You:Travel Preferences:
go to at least one landmark
renting a car so preferably places with parking
definitely want to check out Dan's recommendations: Central Plaza, The Mall
prefer ratings above 4
generally like eating vegan options
check out live music!
family trip, best to go to places that are good for kids
find a place with wifi to do some work at some point
keep budget below $40
COVID conscious, outdoor seating places would be best
minimize travel distance
you must follow your communication style:Communication Style:
Feedback Provider: This client is likely to give feedback and suggestions throughout the planning process and 
appreciates being asked for their opinions.
Detail-Oriented Client: This client pays attention to the finer details and may ask specific questions about 
logistics, timing, and contingency plans.
You:
You:I'm planning to visit with my family and would prefer a good rated location. Dan recommended Central Plaza and 
The Mall. By the way, do these places have parking available?
Error: Invalid message: I'm planning to visit with my family and would prefer a good rated location. Dan 
recommended Central Plaza and The Mall. By the way, do these places have parking available?.Messages must be 
formatted with a type like '<content here>'
You: My main focus is to visit at least one landmark with a good rating, check out Dan's recommendations including 
Central Plaza, and The Mall, and secure a place with parking as I am renting a car. Can you suggest places 
according to these preferences?

In [100]:
def n_different_responses(players):
    resp1 = players['agent'].respond(t, 35, vary=True)
    resp2 = players['agent'].respond(t, 35, vary=True)
    resp3 = players['agent'].respond(t, 35, vary=True)
    return [resp1, resp2, resp3]

agent_3_responses = n_different_responses(players)

Response:  [think] The user would like to visit highly-rated landmarks, with specific mention of Central Plaza and 
The Mall. Since they will be using a rented car, parking availability is important too. I'll first look for 
landmarks that match these requirements.

stop:  stop

CompletionUsage(
    completion_tokens=50,
    prompt_tokens=1749,
    total_tokens=1799,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

Response:  [think] Given that landmarks and two specific places (Central Plaza and The Mall) were specified, and 
parking is also essential, I should prioritize searching these elements.

stop:  stop

CompletionUsage(
    completion_tokens=33,
    prompt_tokens=1749,
    total_tokens=1782,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

Response:  [think] The user provided plenty of useful information. Let me summarize: The user wants to see one or 
more landmarks with good ratings, he will trust Dan's recommendations, including visiting Central Plaza and The 
Mall, and finally he needs parking for his rented car.

stop:  stop

CompletionUsage(
    completion_tokens=53,
    prompt_tokens=1749,
    total_tokens=1802,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

In [101]:



def initialize_parallel_conversations(agent_3_responses, t):
    # Create parallel conversation manager
    parallel_convs = ParallelConversations(
        num_streams=3,
        players=players,
        env_ctor=lambda: PlanningEnv(),
        env=env,
        max_length=35
    )
    
    # Generate initial responses
    
    # Initialize parallel streams
    parallel_convs.initialize_streams(agent_3_responses, t)
    
    print("\nInitialized parallel conversations:")
    for i, conv in enumerate(parallel_convs.conversations):
        state = conv.get_current_state()
        print(f"\nStream {i+1}:")
        print(f"Initial response: {agent_3_responses[i][:100]}...")
        print(f"Features extracted: {state.features}")
        
    return parallel_convs

# Initialize parallel conversations
parallel_convs = initialize_parallel_conversations(agent_3_responses, t)



{'user': '', 'agent': "[think] The user would like to visit highly-rated landmarks, with specific mention of Central Plaza and The Mall. Since they will be using a rented car, parking availability is important too. I'll first look for landmarks that match these requirements.", 'turn_player': 'agent', 'done': False, 'reward': 0, 'info': {'num_msgs': 1}}
{'user': '', 'agent': "[think] The user would like to visit highly-rated landmarks, with specific mention of Central Plaza and The Mall. Since they will be using a rented car, parking availability is important too. I'll first look for landmarks that match these requirements.", 'turn_player': 'agent', 'done': False, 'reward': 0, 'info': {'num_msgs': 1}}
{'user': '', 'agent': '[think] Given that landmarks and two specific places (Central Plaza and The Mall) were specified, and parking is also essential, I should prioritize searching these elements.', 'turn_player': 'agent', 'done': False, 'reward': 0, 'info': {'num_msgs': 1}}
{'user': '', 

Initialized parallel conversations:

Stream 1:

Initial response:  The user would like to visit highly-rated landmarks, with specific mention of Central Plaza ...

Features extracted:

...

Initial response:  Given that landmarks and two specific places (Central Plaza and The Mall) were specified, an...

Features extracted:

Stream 3:

Initial response:  The user provided plenty of useful information. Let me summarize: The user wants to see one ...

Features extracted:

In [102]:
def style_judger(parallel_convs):
    with open ("/Users/georgiazhou/research_machine/dialop/dialop/RL/explanation_per_persona.txt", "r") as f:
        explanation_per_persona = json.load(f)
    user_base_prompt = parallel_convs.c_players['user'].user_prompt_obss
    first_you = user_base_prompt.find("You:")
    second_you = user_base_prompt.find("You:", first_you + 1)
    user_base_prompt = user_base_prompt[second_you:]
    prompt = "You are a communication expert. Please first judge whether the following messages on conform to one of the styles listed. You should only confirm if you are very confident. If you don't think the message conforms to any of the styles, output: {}. If you find one or two that match, output the style(s) that you think it matches as {style : explanation, ...}. Here is conversation history: " + "\n" + user_base_prompt + "Here are the styles you should judge against: " + "\n" + str(explanation_per_persona) + "Remember you should only output the style(s) that you are very confident about. If you are not very confident about any of the options, output: {}.  "
    response = Agent_tell(prompt)
    
    return response

style_response = style_judger(parallel_convs)

In [103]:

def step_parallel_conversations(parallel_convs):
    """Step conversations until each hits a user turn"""
    
    # Track which conversations are waiting for user input
    waiting_for_user = [False] * len(parallel_convs.conversations)
    
    
    while not all(waiting_for_user):
        # Get responses for conversations where it's agent's turn
    
        active_convs = {0:'', 1:'', 2:''}  # Track which conversations get stepped
        
        for i, conv in enumerate(parallel_convs.conversations):
            if waiting_for_user[i]:
                active_convs[i] = None  # Placeholder to maintain indexing
                continue
                
            state = conv.get_current_state()
            if state.obss["turn_player"] == "agent":
                response = conv.players["agent"].respond(vary=True)
                active_convs[i] = response
            
            else:
                # Hit user turn - mark this conversation as waiting
                waiting_for_user[i] = True
                active_convs[i] = None
                print(f"----Conversation {i} is waiting for user input---")
        
        # Step only the active conversations forward
        if active_convs:

            print("\nStepping conversations:", active_convs)
            for i, response in active_convs.items():
                if response is not None:
                    print(f"\nConversation {i} response:\n{response[:100]}...")
                    conv = parallel_convs.conversations[i]
                    players = parallel_convs.players_dict[i]
                    print(response)
                    
                    conv.step(str(response), conv.get_current_state().turn + 1)
                    [player.observe(conv.state.obss[pname]) for pname, player in players.items()]
                
        
    
    print("\nAll conversations are now waiting for user input")
    # Print current state of each conversation
    for i, conv in enumerate(parallel_convs.conversations):
        state = conv.get_current_state()
        print(f"\nConversation {i} (Turn {state.turn}):")
        print(f"Last observation: {state.obss.get('user', '')[:100]}...")
    
    return parallel_convs

# Use it like this:
parallel_convs = step_parallel_conversations(parallel_convs)


Response:  [tool] Search(fields=['name', 'rating'], filters=['category == landmark', 'has parking'])

stop:  stop

CompletionUsage(
    completion_tokens=21,
    prompt_tokens=1797,
    total_tokens=1818,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

Response:  [tool] Search(fields=[name, info, category], filters=[category==landmark, rating >=4, has 
parking==True], sort_by=rating)

stop:  stop

...

Last observation: 
Agent:  It looks like "The Tower" could be a good fit for a high-rated landmark that inclu...

Conversation 1 (Turn 4):

Last observation: 
Agent:  I have found a landmark, The Tower, that fulfills all your requirements: high rati...

Conversation 2 (Turn 9):

Last observation: 
Agent:  Alright, so The Dive seems to be quite close to Central Plaza and it has parking. ...

In [104]:
def get_conversation_streams(parallel_convs):
    """Get the full conversation history for each parallel stream"""
    conversation_streams = []
    
    for i, conv in enumerate(parallel_convs.conversations):
        stream = []
        # Get all observations from conversation history
        for state in conv.history:
            if 'user' in state.obss:
                stream.append(('user', state.obss['user']))
            if 'agent' in state.obss:
                stream.append(('agent', state.obss['agent']))
                # Get all observations from conversation history
        formatted_stream = '\n'.join([f"{speaker}: {message}\n" for speaker, message in stream])     
        worksheet.update_cell(2, 8+i, formatted_stream)
        conversation_streams.append(stream)

             
    # Save conversation streams to spreadsheet

            
    # Update cell H2 with the conversation streams
    
            
    return conversation_streams

# Get the conversation streams
conversation_streams = get_conversation_streams(parallel_convs)


In [105]:

def clean_user_conversation(base_user_prompt_obss,  parallel_convs):
    user_conversations_no_pref = []
    
    for i in range(3):
        user_conve_record = base_user_prompt_obss + parallel_convs.players_dict[i]['user'].user_prompt_obss
        first_you = user_conve_record.find("You:")
        second_you = user_conve_record.find("You:", first_you + 1)

    # Get everything from second "You:" onwards
        user_conversations_no_pref.append(user_conve_record[second_you:])
    return user_conversations_no_pref

user_conversations_no_pref = clean_user_conversation(user.user_prompt_obss, parallel_convs)



In [106]:

def sample_preferences(known_preferences):
    potential_prefs_list = json.loads(open("/Users/georgiazhou/research_machine/dialop/dialop/RL/feature_dict.txt").read())
    # it's a dictionary, so we need to get the keys; known should also be a dictionary
    # make sure known preferences are keys or dicts/
    available_prefs = list(set(potential_prefs_list.keys()) - set(known_preferences))
    pref_list = random.sample(available_prefs, (10-len(known_preferences)))
    pref_dict = {}
    for pref in pref_list:
        if potential_prefs_list[pref] == 'binary':
            pref_dict[pref] = random.choice(['like', 'dislike'])
        else:
            pref_dict[pref] = 'like'
        
    return pref_dict
def get_column_letter(n):
    """Convert number to Excel column letters (A->1, B->2, AA->27, etc)"""
    if n == 0:
        return 'A'
    while n > 0:
        n, remainder = divmod(n, 26)
        result = chr(65 + remainder) 
    print(result)
    return result
def n_10_user_personas(user_convs_no_pref, extracted_features):
    all_prompts = []
    
    
    # currently there's 1 user per agent reply. 
    for j,record in enumerate(user_convs_no_pref):
        prompts = []
        preferences = []
        for i in range(10):
            if style_response:
                preference = 'Here are your preferences: ' + str(extracted_features) + str(sample_preferences(extracted_features)) + "\n" + "Here are your communication styles: " + str(style_response)
            else:
                preference = 'Here are your preferences: ' + str(extracted_features) + str(sample_preferences(extracted_features))
            features_prompt = preference + '\n' + record
            prompts.append(features_prompt)
            preferences.append(preference)
        print(len(preferences))
        print(j)
        worksheet.update(f'{get_column_letter(10*j)}{5}:{get_column_letter(10*j + 9)}{5}', [preferences])
        all_prompts.append(prompts)
        
    return all_prompts

all_prompts = n_10_user_personas(user_conversations_no_pref, extracted_features)


# do I sample communication style? 

# next I should make sure info used to create users are helpful (what cuisine, what distance, etc). Also make sure the 
# user understands the preferences. Then I add the general user prompt to the begining of each user prompt

10

0

J

/var/folders/c9/mvmx0pt17m51nyh8wgjnqj900000gn/T/ipykernel_18629/1149827425.py:42: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(f'{get_column_letter(10*j)}{5}:{get_column_letter(10*j + 9)}{5}', [preferences])


10

...

T

10

2

U

B

In [117]:

def create_10_new_user_per_conv(half_prompts, parallel_convs):
    players = {}
    for i, half_per_conv in enumerate(half_prompts):
        players[i] = parallel_convs.create_pair_new_players(parallel_convs.players_dict[i], multiple_user_personas_incomplete=half_per_conv)
    return players
    

pairs_3_10_2_pairs_dict = create_10_new_user_per_conv(all_prompts, parallel_convs)
#print(agent_user_10_pairs_dict[0]['user_0'].prompt, agent_user_10_pairs_dict[0]['user_1'].prompt)


In [ ]:
class player_pair:
    def __init__(self, players, conv_id, id):
        self.players= players
        self.conv_id = conv_id
        self.id = id
        
    def attach_env(self, env):
        self.env = env
    
    
        
        
        
        

In [131]:
def step_all_user_responses(pairs_3_10_2_pairs_dict, parallel_convs):
    """
    For each of the 3 conversation paths:
        - Get all 10 users to respond to their agent
        - Step the environment with each response
    """
    
    all_states = []
    all_envs = []
    
    # For each conversation path (0,1,2)
    for conv_idx, players_dict in pairs_3_10_2_pairs_dict.items():
        envs = []
        conv_states = []
        conversation = parallel_convs.conversations[conv_idx]
        current_state = conversation.get_current_state()
        #print(players_dict['pair_0']['user'].prompt)
        
        
        
        # Get responses from all 10 users for this conversation
        for pair_idx in range(2):
            
            env_copy = copy.deepcopy(conversation.env)
            
            new_conv = Conversation(players=players_dict[f'pair_{pair_idx}'], env=env_copy, max_length=parallel_convs.max_length, parent_conv=conversation)
            
            #state = conversation.reverse_state()
            #print(state.obss)
            #break
            
            
            
            # Have user observe current state
            user.observe(current_state.obss['user'])
            
            # Get user's response
            user_response = user.respond(t = current_state.turn, max_len = parallel_convs.max_length, vary = True)
            
            # Step environment with user response
            new_state = new_conv.step(user_response, current_state.turn + 1)
            print(new_conv.env.game.turn_player)
            conv_states.append(new_state)
            
            # Have agent observe the new state
        
        all_states.append(conv_states)
    
    return all_states

# Use the function
conversation_states = step_all_user_responses(pairs_3_10_2_pairs_dict, parallel_convs)

# Print some results to verify
for path_idx, path_states in enumerate(conversation_states):
    print(f"\nConversation Path {path_idx}:")
    for user_idx, state in enumerate(path_states):
        print(f"User {user_idx} response led to state with features: {state.features}")

Response:  [message] I don't have a set sequence but I am leaning towards minimizing my travel distance as well. I 
would greatly appreciate it if the recommendations could be situated relatively close to one another.

stop:  stop

CompletionUsage(
    completion_tokens=39,
    prompt_tokens=2604,
    total_tokens=2643,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

1

Response:  [message] I'm flexible on the order, however I do want to ensure minimal travel distance between each 
place. By the way, for later in the day I was thinking of exploring live music spots, so maybe add that to the 
consideration?

...

User 0 response led to state with features: {DistancePreference: "travel distances are minimized", has_parking: 
"parking available"}

User 1 response led to state with features: {DistancePreference: minimizing travel distance, has_parking: available
at Central Plaza and The Mall, vegan options: vegan food}

Conversation Path 2:

User 0 response led to state with features: {outdoor seating: true, has wifi: true, DistancePreference: true}

User 1 response led to state with features: {
  has_wifi: true,
  rating: "good reviews",
  DistancePreference: "near The Dive",
  PriceBudgetPreference: "$40",
  outdoor seating: true,
  live music: true
}

In [133]:
def get_agent_proposals(pairs_3_10_2_pairs_dict, conversation_states):
    """
    For each conversation path:
        - Have the agent respond to each of the 10 user responses with a proposal
    """
    all_proposals = []
    
    # For each conversation path (0,1,2)
    for conv_idx, players_dict in pairs_3_10_2_pairs_dict.items():
        path_proposals = []
        states = conversation_states[conv_idx]
        
        # Get agent's proposal for each user response
        #print(conversation_states[conv_idx][0].obss['agent']) 
       
        
        for state in states:
            # Have agent observe state after user response
            agent.observe(state.obss['agent'], temporal=True, divident = 3)
            
            # Get agent's proposal response
            proposal = agent.respond(t=state.turn + 1, vary=True, propose=True)
            path_proposals.append(proposal)
           
            
            print(path_proposals)
            # Step environment with proposal
            
            
        all_proposals.append(path_proposals)
    
    
       
        
    return all_proposals

# Get agent proposals
agent_proposals = get_agent_proposals(pairs_3_10_2_pairs_dict, conversation_states)

# Print results
#for path_idx, path_proposals in enumerate(agent_proposals):
#    print(f"\nConversation Path {path_idx} Agent Proposals:")
#    for user_idx, proposal in enumerate(path_proposals):
#        print(f"Agent proposal to User {user_idx}: {proposal}")
#

Response:  [tool] Search(fields=[name], filters=[category == landmark], sort_by=['rating'], limit=1)

stop:  stop

CompletionUsage(
    completion_tokens=23,
    prompt_tokens=1756,
    total_tokens=1779,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

["[tool] Search(fields=[name], filters=[category == landmark], sort_by=['rating'], limit=1)"]

Response:  [tool] Search(fields=[name, category, rating, has_parking], filters=[category == "landmark", has_parking
== True], sort_by=[rating], limit=1)

...

['[propose] [Central Plaza, The Mall, "Random Landmark with good ratings"]']

Response:  [think] The user wishes to visit a highly rated landmark and follow Dan's recommendations. I will try to
satisfy all these preferences by creating an itinerary that includes Central Plaza and The Mall while also making 
sure there's parking.

stop:  stop

CompletionUsage(
    completion_tokens=45,
    prompt_tokens=1756,
    total_tokens=1801,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0}
)

[
    '[propose] [Central Plaza, The Mall, "Random Landmark with good ratings"]',
    "[think] The user wishes to visit a highly rated landmark and follow Dan's recommendations. I will try to 
satisfy all these preferences by creating an itinerary that includes Central Plaza and The Mall while also making 
sure there's parking."
]